In [1]:
%%capture
import sys
sys.path.append("../../bus_service_increase")
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd

from IPython.display import display, HTML

from bus_service_utils import chart_utils
from calitp_data_analysis import calitp_color_palette as cp
from update_vars import GCS_FILE_PATH, PUBLIC_FILENAME, YEAR, MONTH
from monthly_ridership_by_rtpa import sum_by_group
#from shared_utils.rt_dates import MONTH_DICT

#alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

In [2]:
# parameters cell for local
rtpa = "Metropolitan Transportation Commission"

In [3]:
%%capture_parameters
rtpa

{"rtpa": "Metropolitan Transportation Commission"}


# {rtpa}
## Annual Ridership Trends

**Download data from our public [folder](https://console.cloud.google.com/storage/browser/calitp-publish-data-analysis)** by navigating to `ntd_annual_ridership` and selecting a file.

Transit operators/agencies that aubmit annual reports to NTD are included in this report. Reporters that were previously Urban full reporters, but are currently not, may appear. This may result in Reporters showing zero or partial ridership data in the report. 

If a Reporter is not a monthly reporter, or has not reported data since 2018, they will not appear in the report.

Examples: 
- **Reporter A** is an urban full reporter from 2019-2022, then became a reduced reporter for 2023. Reporter A's ridership data will be displayed for 2019-2022 only.
- **Reporter B** is an urban full reporter from 2000-2017, then became a reduced reporter for 2018. Reporter B will be named in the report, but will not display ridership data.
- **Reporter C** was a reduced reporter form 2015-2020, then became an urban full reporter and began submitting monthly ridership data to NTD for 2021. Reporter C's ridership data will be displayed for 2021-present. 

In [4]:
URL = ("https://console.cloud.google.com/storage/"
       "browser/calitp-publish-data-analysis"
      )

display(
    HTML(
        f"""
        <a href={URL}>
        Download the latest month of data: {PUBLIC_FILENAME}</a>
        """
    )
)

In [5]:
#read in data
df = pd.read_parquet(
    f"{GCS_FILE_PATH}annual_report_data_2022-2023.parquet"
).rename(
    columns={
       "actual_vehicles_passenger_car_revenue_hours":"vrh",
       "actual_vehicles_passenger_car_revenue_miles":"vrm",
       "unlinked_passenger_trips_upt":"upt" 
    }
)

In [10]:
def sum_by_group(
    df: pd.DataFrame,
    group_cols: list) -> pd.DataFrame:
    """
    since data is now long to begin with, this replaces old sum_by_group, make_long and assemble_long_df functions.
    """
    grouped_df = df.groupby(group_cols
                           ).agg({
        "upt":"sum",
        "previous_y_m_upt":"sum",
        "change_1yr":"sum"
    }
    ).reset_index()
    
    #get %change back
    grouped_df = get_percent_change(grouped_df)
    
    #decimal to whole number
    grouped_df["pct_change_1yr"] = grouped_df["pct_change_1yr"]*100
    
    return grouped_df

In [13]:
## this also needs `get_percent_change()` and `add_change_columns()` function. 
## may move all these functions to a script file

In [11]:
def group_by_agency(df):
    """
    Take in the 'by_ageny_long' df and aggregatese by rtpa, and calculates upt % of total. 
    To be used in pie chart
    """
    initial_agg = df.groupby("agency").agg(
        total_upt=("upt","sum")
    ).reset_index()
    
    # % total columns
    initial_agg["pct_of_total_upt"] = (initial_agg["total_upt"]/initial_agg["total_upt"].sum())*100
    
    # cleaning data types and rounding
    initial_agg["total_upt"] = initial_agg["total_upt"].astype("int64")
    initial_agg["pct_of_total_upt"] = initial_agg["pct_of_total_upt"].round(decimals=2)
    cleaned_agg = initial_agg.sort_values(by="total_upt", ascending = False)
    
    return cleaned_agg

In [12]:
agency_cols = ["ntd_id", "agency", "RTPA"]

by_agency_long = sum_by_group(df, agency_cols)

# agg by agency, for pie chart
agency_agg_yr = group_by_agency(by_agency_long)

# total UPT check 
total_upt = by_agency_long["upt"].sum()
agency_count = by_agency_long["agency"].nunique()

KeyError: "Column(s) ['change_1yr', 'previous_y_m_upt'] do not exist"

In [ ]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-06"])
              ]    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = labeling(y_col)),
             color = alt.Color(color_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
        
    return chart

In [ ]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
            "pct_change_1yr":"Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023 
    #df = df.dropna(subset = y_col)
    
    #need flag for y_col >,<, 0, missing?
    #count function to how many agencies fall in those categories, then look at those agencies
    #present table
    
    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-03", "-06", "-09"])
              ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = short_label(y_col)),
             color = alt.Color(color_col, title = "", 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
    
    return chart

In [ ]:
def make_pie_chart(df,
             col,
             color_col):
    
    pie = alt.Chart(df).mark_arc(radius = 150).encode(
        theta=col,
        color=color_col,
        tooltip = ["agency","total_upt","pct_of_total_upt"]
    ).properties(
        title=f"Total Unlinked Passenger Trips per Reporter in RTPA since {MIN_YEAR}",

    )

    return pie

In [ ]:
#simple bar chart for total agencies and UPT
def total_upt_chart(df: pd.DataFrame, x_col:str, y_col:str,tool_tip:list):
    bar_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(x_col).sort("-y"),
    y=alt.Y(y_col),
    tooltip = tool_tip,
    color = alt.Color(x_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               ))
    ).properties(
    title=f"Total Unlinked Passenger Trips per Reporter in RTPA since {MIN_YEAR}",
    width = WIDTH, 
    height = HEIGHT,
    ).resolve_scale(y="independent").interactive()
    
    return bar_chart

### Report Totals

In [ ]:
Markdown(f"""
Within {rtpa}:
- Number of Reporters: <b>{agency_count}</b>.
- Total Unlinked Passenger Trips since {MIN_YEAR}: <b>{total_upt:,}</b>.
- Individual Reporters ridership breakdown:
""")

In [ ]:
display(agency_agg_yr.reset_index(drop=True))

In [ ]:
tooltip_list = ["agency","total_upt","pct_of_total_upt"]

total_upt_chart(
    agency_agg_yr,
    x_col="agency", 
    y_col="total_upt", 
    tool_tip=tooltip_list)

### Reporter

In [ ]:
make_line_chart(
    by_agency_long, 
    y_col = "upt", 
    color_col = "agency")


In [8]:
make_bar_chart(
    by_agency_long, 
    y_col = "change_1yr", 
    color_col = "agency")

NameError: name 'make_bar_chart' is not defined